In [22]:
import gurobipy as gp

def build_instance(file_name):
    print(f"BUILDING INSTANCE {file_name.split("/")[len(file_name.split("/"))-1]}")
    instance_file = open(file_name,"r")
    
    lines = [line.strip() for line in instance_file.readlines()]
    current_line = 0
    
    #Salvando quantidade de clientes e quantidade e lockers
    qty_customer = int(lines[current_line].split(" ")[0])
    qty_locker = int(lines[current_line].split(" ")[1])
    
    current_line += 1
    
    max_vehicle = int(lines[current_line].split(" ")[0])
    vehicle_capacity = float(lines[current_line].split(" ")[1])
    
    current_line += 1
    
    demands = list()
    ## Salvando demands de clients
    for demand in range(current_line,qty_customer+2):
        demands.append(float(lines[demand]))
    
    current_line += qty_customer
    
    #Preenchendo nos
    nodes = dict()
    
    #Deposito
    depot = {
        'label': "D",
        'x' : float(lines[current_line].split(" ")[0]),
        'y' : float(lines[current_line].split(" ")[1]),
        'earliest' : float(lines[current_line].split(" ")[2]),
        'latest' : float(lines[current_line].split(" ")[3]),
        'service_time' : float(lines[current_line].split(" ")[4]),
        'demand': 0,
        'type' : int(lines[current_line].split(" ")[5])
    }
    nodes['D']=depot
    current_line += 1

    #Clientes
    customers = dict()
    customer_count = 1
    for i in range(current_line, current_line + qty_customer):
        customer = {
        'label' : f"C{customer_count}",
        'x' : float(lines[i].split(" ")[0]),
        'y' : float(lines[i].split(" ")[1]),
        'earliest' : float(lines[i].split(" ")[2]),
        'latest' : float(lines[current_line].split(" ")[3]),
        'service_time' : float(lines[i].split(" ")[4]),
        'demand': demands[customer_count-1],
        'type' : int(lines[i].split(" ")[5])
        }
        customer_count+=1
        customers[customer['label']]=customer
        nodes[customer['label']]=customer
    current_line += qty_customer

    #Lockers
    lockers = dict()
    locker_count = 1
    for i in range(current_line,current_line+qty_locker):
        locker = {
        'label' : f"P{locker_count}",
        'x' : float(lines[i].split(" ")[0]),
        'y' : float(lines[i].split(" ")[1]),
        'earliest' : float(lines[i].split(" ")[2]),
        'latest' : float(lines[current_line].split(" ")[3]),
        'service_time' : float(lines[i].split(" ")[4]),
        'demand': 0,
        'customers': list(),
        'type' : int(lines[i].split(" ")[5])
        }
        locker_count+=1
        lockers[locker['label']] = locker
        nodes[locker['label']]=locker
    current_line += qty_locker

    #Atribuição de lockers
    current_customer = 0
    for i in range(current_line, current_line+qty_customer):
        try:
            customer = customers[list(customers.keys())[current_customer]]
            
            locker_index = lines[i].split(" ").index("1")
            locker = lockers[list(lockers.keys())[locker_index]]
            customer['locker'] = locker['label']
            locker['customers'].append(customer['label'])
        except ValueError:    
            customer['locker'] = ""
        current_customer += 1
    current_line += qty_customer
    
    #Matriz de Distancia
    costs = dict()
    for i,node_i in enumerate(nodes):
         current_node_i = nodes[node_i]
         current_node_i['index'] = i
         for j,node_j in enumerate(nodes):
             current_node_j = nodes[node_j]
             distance = ((current_node_i['x']-current_node_j['x'])**2 + (current_node_i['y']-current_node_j['y'])**2)**(1/2)
             costs[current_node_i['label'],current_node_j['label']] = distance
    print(f"Qty customer:{qty_customer}")
    print(f"Qty locker:{qty_locker}")
    print(f"Max vehicles:{max_vehicle}")
    print(f"Vehicle capacity:{vehicle_capacity}")
    print(f"BUILD FINISHED")
    print(sum(demands))
    return {
        'qty_customer': qty_customer,
        'qty_locker': qty_locker,
        'qty_nodes': qty_locker,
        'total_demands': sum(demands),
        'min_vehicles': round(sum(demands)/vehicle_capacity),
        'max_vehicle': max_vehicle,
        'vehicle_capacity': vehicle_capacity,
        'nodes': nodes,
        'customers': customers,
        'lockers': lockers,
        'depot': depot,
        'costs': costs
    }


instance = build_instance("../../instances/vrppl/inst_test")

BUILDING INSTANCE inst_test
Qty customer:10
Qty locker:2
Max vehicles:2
Vehicle capacity:200.0
BUILD FINISHED
150.0


In [24]:
qty_vehicles = 2

# demands = [node['demand'] for node in instance['nodes']]

#Rotulos

#Nos
nodes = instance['nodes']
node_labels = list(nodes.keys())
customer_labels = list(instance['customers'].keys())
locker_labels = [node for node in node_labels if nodes[node]['type'] == 4]
customers_hc = [customer for customer in customer_labels if nodes[customer]['type'] == 1]
customers_lc = [customer for customer in customer_labels if nodes[customer]['type'] == 2]
customers_hlc = [customer for customer in customer_labels if nodes[customer]['type'] == 3]


#Veiculos
vehicles = list()
for i in range(qty_vehicles):
    vehicles.append(f"v_{i}")





In [42]:
#Criando dicts

#Custos
costs = instance['costs']

#Atribuição de locker l_ij i in N_C j in N_PL
lockers_attrib = dict();
for customer in customer_labels:
    for locker in locker_labels:
        lockers_attrib[customer,locker] = int(nodes[customer]['locker'] == locker)
        
print(lockers_attrib)

#Criando modelo
m = gp.Model()
m.setParam(gp.GRB.Param.OutputFlag,0)

#Inserindo variaveis
x_ijk = m.addVars(node_labels, node_labels,vehicles, vtype = gp.GRB.BINARY)
h_i = m.addVars(customer_labels, vtype = gp.GRB.INTEGER)
l_i = m.addVars(customer_labels, vtype = gp.GRB.INTEGER)
y_ij = m.addVars(customer_labels, locker_labels, vtype = gp.GRB.INTEGER) #i=cliente j=locker
print('\n',y_ij)
# psi_jk

{('C1', 'P1'): 0, ('C1', 'P2'): 1, ('C2', 'P1'): 1, ('C2', 'P2'): 0, ('C3', 'P1'): 0, ('C3', 'P2'): 1, ('C4', 'P1'): 0, ('C4', 'P2'): 1, ('C5', 'P1'): 0, ('C5', 'P2'): 1, ('C6', 'P1'): 0, ('C6', 'P2'): 1, ('C7', 'P1'): 0, ('C7', 'P2'): 0, ('C8', 'P1'): 0, ('C8', 'P2'): 1, ('C9', 'P1'): 0, ('C9', 'P2'): 0, ('C10', 'P1'): 0, ('C10', 'P2'): 0}

 {('C1', 'P1'): <gurobi.Var *Awaiting Model Update*>, ('C1', 'P2'): <gurobi.Var *Awaiting Model Update*>, ('C2', 'P1'): <gurobi.Var *Awaiting Model Update*>, ('C2', 'P2'): <gurobi.Var *Awaiting Model Update*>, ('C3', 'P1'): <gurobi.Var *Awaiting Model Update*>, ('C3', 'P2'): <gurobi.Var *Awaiting Model Update*>, ('C4', 'P1'): <gurobi.Var *Awaiting Model Update*>, ('C4', 'P2'): <gurobi.Var *Awaiting Model Update*>, ('C5', 'P1'): <gurobi.Var *Awaiting Model Update*>, ('C5', 'P2'): <gurobi.Var *Awaiting Model Update*>, ('C6', 'P1'): <gurobi.Var *Awaiting Model Update*>, ('C6', 'P2'): <gurobi.Var *Awaiting Model Update*>, ('C7', 'P1'): <gurobi.Var *Awa

In [50]:
#Inserindo restrição
c2 = m.addConstrs(
    gp.quicksum(x_ijk[node_i,node_j,vehicle] for vehicle in vehicles for node_j in node_labels) == 1  for node_i in customers_hc
)

c3 = m.addConstrs(
    gp.quicksum(x_ijk[node_i,node_j,vehicle] for node_j in node_labels for vehicle in vehicles ) <= h_i[node_i] for node_i in customer_labels
)

c4 = m.addConstrs(
    gp.quicksum(x_ijk[node_i,node_j,vehicle] for vehicle in vehicles for node_j in node_labels) == 0  for node_i in customers_lc
)

c5 = m.addConstrs(
    gp.quicksum(x_ijk["D",node_j,vehicle] for node_j in customer_labels) <= 1 for vehicle in vehicles
)

c6 = m.addConstrs(
    (gp.quicksum(x_ijk[node_i,node_j,vehicle] for node_i in node_labels if node_i!=node_j) - gp.quicksum(x_ijk[node_j,node_i,vehicle] for node_i in node_labels  if node_i!=node_j)) == 0 for vehicle in vehicles for node_j in node_labels 
)

c7 = m.addConstrs(
    gp.quicksum(x_ijk[node_i, "D",vehicle] for node_i in customer_labels) <= 1 for vehicle in vehicles
)

c8 = m.addConstrs(
    gp.quicksum(nodes[node_j]['demand']*x_ijk[node_i,node_j,vehicle] for node_i in node_labels for node_j in customer_labels) <= instance['vehicle_capacity'] for vehicle in vehicles
)

#---#

c13 = m.addConstrs(
    h_i[node_i] + l_i[node_i] == 1 for node_i in customer_labels
)

c14 = m.addConstrs(
    h_i[node_i] == 1 for node_i in customers_hc
)

c15 = m.addConstrs(
    l_i[node_i] == 1 for node_i in customers_lc
)

#---#
c17 = m.addConstrs(
    y_ij[customer_i,locker_j] <= lockers_attrib[customer_i,locker_j] for customer_i in customer_labels for locker_j in locker_labels
)


#Definindo função objetivo
m.setObjective(
    gp.quicksum(costs[node_i,node_j] * x_ijk[node_i,node_j,vehicle] for node_i in node_labels for node_j in node_labels for vehicle in vehicles),
    sense = gp.GRB.MINIMIZE
)


print(f"Valor Total: {m.objVal}\n")
# print(m)
# print(m.getObjective())

for vehicle in vehicles:
    print(vehicle)
    for node_i in node_labels:
        for node_j in node_labels:
            if round(x_ijk[node_i,node_j,vehicle].X) == 1:
                print(f"{node_i},{node_j}", " ", end="")
                
    print('\n')
    
print("Variavel h_i")
for customer in customer_labels:
    print(round(h_i[customer].X), end=" ")

print("\nVariavel l_i")
for customer in customer_labels:
    print(round(l_i[customer].X), end=" ")

print("\nVariavel y_ij")
for customer in customer_labels:
    for locker in locker_labels:
        print(round(y_ij[customer,locker].X), end=" ")


Valor Total: 0.0

v_0
D,D  P1,P1  P2,P2  

v_1
D,D  C7,C7  C9,C9  C10,C10  P1,P1  P2,P2  

Variavel h_i
0 0 0 1 1 1 1 0 1 1 
Variavel l_i
1 1 1 0 0 0 0 1 0 0 
Variavel y_ij
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 